<a href="https://colab.research.google.com/github/Yogasri-2005/Yogasri-Datascience/blob/main/LEX_and_YACC_Compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEX and YACC Compiler in Colab

Drawbacks:
* Regular interrupts (Ctrl+D, Ctrl+C) for shell won't work in Colab while inputting for program.
<br>Workaround: Store your inputs in a txt file and pass it to the program.

In [43]:
#@title Install *prerqeuisites* (run this cell first to work on LEX/YACC)
!sudo apt install flex bison

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
bison is already the newest version (2:3.8.2+dfsg-1build1).
flex is already the newest version (2.6.4-8build2).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


## Lex only

In [44]:
#@title Writing Lex program
%%writefile program.l

%{
    #include <stdio.h>
    int ctChar=0;
    int ctSpace=0;
    int ctWord=0;
    int ctLine=0;
%}
WORD [^ \t\n,\.:]+
EOL [\n]
BLANK [ ]
%%

{WORD} {ctWord++; ctChar+=yyleng;}
{BLANK} {ctSpace++;}
{EOL} {ctLine++;}
. {ctChar++;}
%%

void main(int argc, char *argv[]){
    if(argc!=2){
        printf("Usage:\n\t./a.out <FILENAME>\n");
        exit(0);
    }

    yyin=fopen(argv[1],"r");
    yylex();

    printf("Word Count: %d\n",ctWord);
    printf("Character Count: %d\n",ctChar);
    printf("Space Count: %d\n",ctSpace);
    printf("Line Count: %d\n",ctLine);
    fclose(yyin);

}

int yywrap(){
    return 1;
}

Overwriting program.l


if you want to use at txt as an input

In [45]:
%%writefile program.txt

This is a sample file.

Overwriting program.txt


In [46]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

lex -l program.l
gcc lex.yy.c
./a.out program.txt

Word Count: 5
Character Count: 18
Space Count: 4
Line Count: 2


## Lex and Yacc combined

In [47]:
#@title Writing YACC program
%%writefile program.y
%{
#include <stdio.h>
#include <stdlib.h>
#include <string.h>

void yyerror(const char* s) {
    fprintf(stderr, "Parse error: %s\n", s);
}

int yylex(); // Lexer declaration

enum Type { TYPE_INT, TYPE_FLOAT, TYPE_CHAR };
%}

%union {
    int iVal;
    float fVal;
    char* sVal;
    int type; // Used to return the type from expression
}

%token <iVal> INTEGER
%token <fVal> FLOAT
%token <sVal> CHAR
%token EOL

%type <type> expression statement

%%

program:
      /* empty */
    | program line
;

line:
      statement EOL {
        if ($1 == TYPE_INT) {
            printf("Type: INTEGER\n");
        } else if ($1 == TYPE_FLOAT) {
            printf("Type: FLOAT\n");
        } else if ($1 == TYPE_CHAR) {
            printf("Type: CHAR/STRING\n");
        } else {
            printf("Invalid type\n");
        }
    }
;

statement:
    expression {
        $$ = $1;
    }
;

expression:
      INTEGER {
        $$ = TYPE_INT;
    }
    | FLOAT {
        $$ = TYPE_FLOAT;
    }
    | CHAR {
        $$ = TYPE_CHAR;
    }
;

%%

int main() {
    printf("Enter values (integer, float, or char), one per line:\n");
    yyparse();
    return 0;
}



Overwriting program.y


In [48]:
#@title Writing Lex program
%%writefile program.l

%{
#include "y.tab.h"
#include <stdlib.h>
%}

%%
[0-9]+\n        { yylval.iVal = atoi(yytext); return INTEGER; }
[0-9]+\.[0-9]+\n { yylval.fVal = atof(yytext); return FLOAT; }
\"[^\"]*\"\n    { yylval.sVal = strdup(yytext); return CHAR; }
\n              { return EOL; }
[ \t]+          ; // Ignore whitespace
.               { return EOL; } // Handle unexpected chars as EOL
%%

int yywrap() {
    return 1;
}


Overwriting program.l


if you want to use at txt as an input

In [49]:
%%writefile program.txt

This is a sample file.

Overwriting program.txt


In [ ]:
#@title Shell Execution (you can rewrite the commands as per your need, eg. if you want to include a file as an input)
%%shell

yacc -d program.y
lex program.l
cc y.tab.c lex.yy.c -ll
./a.out

Enter values (integer, float, or char), one per line:
